In [1]:
# Standard Packages
import pandas as pd
import pylab as plt
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.interpolate import Rbf, interp1d

# Spatial Handling
import geopandas as gpd
import shapely
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_window
from rasterio.transform import from_origin
from rasterstats import zonal_stats
from affine import Affine
import shapely.geometry
from shapely.geometry import LineString, box, Point
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial.distance import euclidean
from shapely.ops import nearest_points
# import cv2

import math
import glob

import warnings
import os
import subprocess
import tempfile
from pathlib import Path

warnings.filterwarnings(category=FutureWarning, action='ignore')
warnings.filterwarnings(category=DeprecationWarning, action='ignore')
warnings.filterwarnings(category=UserWarning, action='ignore')

### Env variables preprocess

In [ ]:
# concatenate all csv files
csv_folder = r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/env_covariates/WS_daily_data"

pattern = os.path.join(csv_folder, '*.csv')
csv_files = sorted(glob.glob(pattern))

df_list = [pd.read_csv(f) for f in csv_files]

combined_df = pd.concat(df_list, ignore_index=True)

combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')

# combined_df['Day']   = combined_df['Date'].dt.day
combined_df['Month'] = combined_df['Date'].dt.month
combined_df['Year']  = combined_df['Date'].dt.year
combined_df['Day'] = combined_df['Date'].dt.dayofyear
combined_df['Day_cont'] = (combined_df['Year'] - combined_df['Year'].min()) * 365 + combined_df['Day']
# print(combined_df['Year'].min())

In [ ]:
# combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')

filled_frames = []

for station, group in combined_df.groupby('Station_name'):
    df = group.sort_values('Date').copy()
    x = df['Date'].map(pd.Timestamp.toordinal).values
    
    for col in ['Temp_max','Temp_avg','Temp_min','RH_max','RH_min']:
        y = df[col].values
        mask = ~np.isnan(y)
        if mask.sum() < 2:
            continue
        
        # linear interpolater
        # f_linear = interp1d(
        #     x[mask], y[mask],
        #     kind='linear',
        #     bounds_error=False,
        #     fill_value="extrapolate"
        # )
        # df[col] = f_linear(x)
        df[col] = np.interp(x, x[mask], y[mask])
        
    filled_frames.append(df)

interpolated = pd.concat(filled_frames, ignore_index=True)

def F_to_C(F):
    return (F - 32.0) * 5.0/9.0

interpolated['Temp_min_C'] = interpolated['Temp_min'].apply(F_to_C)
interpolated['Temp_avg_C'] = interpolated['Temp_avg'].apply(F_to_C)
interpolated['Temp_max_C'] = interpolated['Temp_max'].apply(F_to_C)

# interpolated.to_csv('new_Maricopa_WS_daily_data1.csv', index=False)

## Create Patches over maricopa county

In [ ]:
class Polygon:
    @staticmethod
    def polygoninitialization(exclude_non_mainland=True, state_init=None, county_name=None):
        # Load US county shapefile
        shapefile_path = r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/Simulated_data/shapefiles/cb_2023_us_county_500k.shp"
        gdf = gpd.read_file(shapefile_path)
        gdf = gdf.to_crs(epsg=4326)

        # Filter for only mainland US states if specified
        if exclude_non_mainland:
            gdf = gdf[(pd.to_numeric(gdf['STATEFP']) < 60) & (pd.to_numeric(gdf['STATEFP']) != 2) & (pd.to_numeric(gdf['STATEFP']) != 15)]

        # Filter by specific state by initial
        if state_init:
            gdf = gdf[gdf['STUSPS'] == str(state_init)]

        # Filter by county
        if county_name:
            gdf = gdf[gdf['NAME'].str.lower() == county_name.lower()]

        gdf['STATEFP'] = gdf['STATEFP'].astype(int)
        gdf = gdf.drop(columns=['LSAD', 'ALAND', 'AWATER'])
        return gdf.reset_index(drop=True)


class Grid:
    @staticmethod
    def create_grid(df, grid_size, mode="default"):
        grid_creator = GridCreator(df, grid_size=grid_size, mode=mode)
        return grid_creator.create_grid()


class GridCreator:
    def __init__(self, gdf, grid_size, overlap=True, crs="EPSG:4326", mode="default"):
        self.gdf = gdf
        self.grid_size = grid_size
        self.overlap = overlap
        self.crs = crs
        self.mode = mode  # New mode parameter to differentiate between methods

    def create_grid(self):
        lon_min, lat_min, lon_max, lat_max = self.gdf.total_bounds

        if self.mode == "square":
            # Ensuring grid_size x grid_size grid when grid_size = 2 → 2x2 grid
            cell_width = (lon_max - lon_min) / self.grid_size
            cell_height = (lat_max - lat_min) / self.grid_size

            grid_cells = [
                box(lon0, lat0, lon0 + cell_width, lat0 + cell_height)
                for lat0 in np.linspace(lat_min, lat_max - cell_height, self.grid_size)[::-1]
                for lon0 in np.linspace(lon_min, lon_max - cell_width, self.grid_size)
            ]

        elif self.mode == "horizontal":
            # split into `grid_size` horizontal strips
            cell_height = (lat_max - lat_min) / self.grid_size
            grid_cells = [
                box(lon_min, lat0, lon_max, lat0 + cell_height)
                for lat0 in np.linspace(lat_min, lat_max - cell_height, self.grid_size)[::-1]
            ]

        else:
            # Default behavior: creates exactly "grid_size" number of cells
            step_size = (lon_max - lon_min) / self.grid_size
            grid_cells = [
                box(lon0, lat_min, lon0 + step_size, lat_max)
                for lon0 in np.linspace(lon_min, lon_max - step_size, self.grid_size)
            ]

        cells = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=self.crs)
        cells['ID'] = range(1, len(cells) + 1)

        # filter grid cells to only those overlapping the area of interest
        if self.overlap:
            cells = cells.sjoin(self.gdf, how='inner').drop_duplicates('geometry')
            cells['ID'] = range(1, len(cells) + 1)

        return cells


class Adjacency_Polygon:
    @classmethod
    def calculate_adjacency_matrix(cls, grid):
        n = len(grid)
        adj_matrix = np.zeros((n, n), dtype=int)

        # Moore neighborhood (grid neighbors that share sides/corners)
        for i in range(n):
            for j in range(i + 1, n):
                if grid.iloc[i].geometry.touches(grid.iloc[j].geometry):
                    adj_matrix[i, j] = 1
                    adj_matrix[j, i] = 1

        return pd.DataFrame(adj_matrix, index=range(1, n + 1), columns=range(1, n + 1))


grid_size = 2

# Load all mainland US
# us_polygon = Polygon.polygoninitialization(exclude_non_mainland=True)

# Create a grid for the US
# grid_us_default = Grid.create_grid(us_polygon, grid_size, mode="default")
# grid_us_2x2 = Grid.create_grid(us_polygon, grid_size, mode="square")

# Load a specific state (e.g., California with FIPS = "06")
county_polygon = Polygon.polygoninitialization(exclude_non_mainland=True, state_init="AZ", county_name='Maricopa')

# Create a grid for Arizona
grid1 = Grid.create_grid(county_polygon, grid_size, mode="horizontal")
# grid_ca_2x2 = Grid.create_grid(county_polygon, grid_size, mode="square")

# Adjacency matrix for a selected grid
adj_matrix = Adjacency_Polygon.calculate_adjacency_matrix(grid1)
# adj_matrix = Adjacency_Polygon.calculate_adjacency_matrix(grid_ca_2x2)

# Display selected grid
# grid_ca_2x2

grid1

### Create maricopa county weather stations shape files

In [ ]:
maricopa_cty_polygon = Polygon.polygoninitialization(exclude_non_mainland=True, state_init="AZ", county_name="Maricopa")

df = pd.read_excel(r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/Simulated_data/data/ALERT_sensors_all_by_name.xlsx")
stations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude_DD, df.Latitude_DD), crs="EPSG:4326")
stations_gdf = stations_gdf.drop(columns=['Dev._Type', 'Installed', 'Device_ID', 'Station_Location'])

keep_df = pd.read_excel(r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/Simulated_data/data/Weather_Stations_names.xlsx")
keep_df["BaseName"] = (keep_df["Station"].str.split(",", n=1).str[0].str.strip())

filt_stations_gdf = stations_gdf[stations_gdf["Station_Name"].isin(keep_df["BaseName"])].copy()
filt_stations_gdf = filt_stations_gdf.drop_duplicates()
filt_stations_gdf = filt_stations_gdf.reset_index(drop=True)

maricopa_stations = gpd.sjoin(filt_stations_gdf, maricopa_cty_polygon, how="left")
maricopa_stations = maricopa_stations.drop(columns=['index_right'])

In [ ]:
ax = maricopa_cty_polygon.plot(fc="none", ec='black', figsize=(6,6))

# Overlay the grid cells
grid1.plot(fc="none", ec='green', ax=ax)
maricopa_stations.plot(fc='none', ec='blue', ax=ax)

for idx, row in grid1.iterrows():
    plt.annotate(text=str(row['ID']), xy=(row['geometry'].centroid.x, row['geometry'].centroid.y), color='red', fontsize=15)
plt.title('Overlay of Grid cells on Maricopa Cunty')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
maricopa_stations1 = gpd.sjoin(filt_stations_gdf, maricopa_cty_polygon, how="inner", predicate="within")
maricopa_stations1 = maricopa_stations1.reset_index(drop=True)

In [ ]:
# # save shape file
# out_gpkg = r"C:\Users\manub\OneDrive - Arizona State University\Research\Research_new\Param_est\Data_and_shape_files\Maricopa_cty_WS_shape_files\Maricopa_WS.gpkg"

# maricopa_cty_polygon.to_file(out_gpkg, layer="boundary", driver="GPKG")
# maricopa_stations1.to_file(out_gpkg, layer="stations", driver="GPKG")

In [ ]:
ax = maricopa_cty_polygon.plot(fc="none", ec='black', figsize=(6,6))

# Overlay the grid cells
grid1.plot(fc="none", ec='green', ax=ax)
maricopa_stations1.plot(fc='none', ec='blue', ax=ax)

for idx, row in grid1.iterrows():
    plt.annotate(text=str(row['ID']), xy=(row['geometry'].centroid.x, row['geometry'].centroid.y), color='red', fontsize=15)
plt.title('Overlay of Grid cells on Maricopa Cunty')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

### Add environmental layers

In [ ]:
env_data = pd.read_csv(r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/Simulated_data/data/new_Maricopa_WS_daily_data1.csv")
env_data['Station_Name'] = (env_data["Station_name"].str.split(",", n=1).str[0].str.strip())
env_data = env_data.drop(columns=['Station_name', 'Day', 'Date'])

# apply a 7-day centered moving average to your RH_min and RH_max
env_data['RH_min_smooth'] = (env_data['RH_min'].rolling(window=7, center=True, min_periods=1).mean())
env_data['RH_max_smooth'] = (env_data['RH_max'].rolling(window=7, center=True, min_periods=1).mean())

# smooth Tmin and Tmax over a 7-day window
env_data['Tmin_smooth'] = (env_data['Temp_min_C'].rolling(window=7, center=True, min_periods=1).mean())
env_data['Tmax_smooth'] = (env_data['Temp_max_C'].rolling(window=7, center=True, min_periods=1).mean())

In [ ]:
stations_env = maricopa_stations1.merge(env_data, on="Station_Name", how="inner")
stations_env = stations_env.drop(columns=['index_right', 'COUNTYNS', 'GEOIDFQ', 'NAMELSAD', 'STATE_NAME'])
# stations_env

In [ ]:
pts_in_grid = gpd.sjoin(stations_env, grid1[['ID','geometry']], how='inner', predicate='within')
cell_means = (pts_in_grid.groupby(['ID', 'Day_cont'])[['Temp_max','Temp_avg','Temp_min','RH_max','RH_min', 'RH_min_smooth', 'RH_max_smooth', 'Temp_max_C',
                'Temp_avg_C', 'Temp_min_C', 'Tmin_smooth', 'Tmax_smooth']].mean().reset_index())
grid_daily = grid1.merge(cell_means, on='ID', how='right')
grid_daily = grid_daily.drop(columns=['index_right', 'COUNTYNS', 'GEOIDFQ', 'NAMELSAD', 'STATE_NAME'])

### Create model Stochastic Petri Net Model (2 Patches)

In [ ]:
# Function to Compute Distance Between Patche centroid for human migration

def haversine_distance(coord1, coord2):
    R = 3959.0 # 6371.0 Earth's radius in km
    lat1, lon1 = map(radians, coord1)
    lat2, lon2 = map(radians, coord2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distance in miles


def compute_distance_matrix(grid, use_haversine=True):
    num_patches = len(grid)
    distance_matrix = np.zeros((num_patches, num_patches))

    for i in range(num_patches):
        for j in range(i + 1, num_patches):
            centroid1 = grid.iloc[i].geometry.centroid
            centroid2 = grid.iloc[j].geometry.centroid
            
            if use_haversine:
                dist = haversine_distance((centroid1.y, centroid1.x), (centroid2.y, centroid2.x))
            else:
                dist = euclidean((centroid1.x, centroid1.y), (centroid2.x, centroid2.y))

            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # Symmetric matrix

    return distance_matrix

In [ ]:
distance_matrix = compute_distance_matrix(grid1, use_haversine=True)
distance_matrix

## Vector patch model

In [ ]:
import xml.etree.ElementTree as ET
# import numpy as np
# import pandas as pd

class SIRModelSBML:
    def __init__(self):
        # Create the root element for the SBML document
        self.root = ET.Element("{http://www.sbml.org/sbml/level3/version1/core}sbml", level="3", version="1")
        self.model = ET.SubElement(self.root, "model", id="Model_generated_by_BIOCHAM")
        self.list_of_compartments = ET.SubElement(self.model, "listOfCompartments")
        self.list_of_species = ET.SubElement(self.model, "listOfSpecies")
        self.list_of_parameters = ET.SubElement(self.model, "listOfParameters")
        self.list_of_initial_assignments = ET.SubElement(self.model, "listOfInitialAssignments")
        self.list_of_reactions = ET.SubElement(self.model, "listOfReactions")

    def add_compartment(self, compartment_id, spatial_dimensions="3", size="1", constant="true"):
        # Add compartment to the model
        ET.SubElement(self.list_of_compartments, "compartment", id=compartment_id,
                      spatialDimensions=spatial_dimensions, size=size, constant=constant)

    def add_species(self, species_id, name, compartment, initial_concentration="0", has_only_substance_units="false",
                    boundary_condition="false", constant="false"):
        # Create species
        ET.SubElement(self.list_of_species, "species", id=species_id, name=name, compartment=compartment,
                      initialConcentration=initial_concentration, hasOnlySubstanceUnits=has_only_substance_units,
                      boundaryCondition=boundary_condition, constant=constant)

    def add_parameter(self, parameter_id, name, value, constant="false"):
        # Create parameter
        ET.SubElement(self.list_of_parameters, "parameter", id=parameter_id, name=name, value=value, constant=constant)

    def add_initial_assignment(self, symbol, math_content):
        # Create initial assignment
        initial_assignment = ET.SubElement(self.list_of_initial_assignments, "initialAssignment", symbol=symbol)
        math = ET.SubElement(initial_assignment, "math", xmlns="http://www.w3.org/1998/Math/MathML")
        math.append(ET.fromstring(math_content))  # Parse MathML from string and append

    def add_reaction(self, reaction_id, reversible="false", reactants=None, products=None, kinetic_law_math=None):
        # Create reaction
        reaction = ET.SubElement(self.list_of_reactions, "reaction", id=reaction_id, reversible=reversible, fast="false")

        # Add reactants
        if reactants:
            list_of_reactants = ET.SubElement(reaction, "listOfReactants")
            for species_id, stoichiometry in reactants.items():
                ET.SubElement(list_of_reactants, "speciesReference", species=species_id, stoichiometry=str(stoichiometry), constant="true")

        # Add products
        if products:
            list_of_products = ET.SubElement(reaction, "listOfProducts")
            for species_id, stoichiometry in products.items():
                ET.SubElement(list_of_products, "speciesReference", species=species_id, stoichiometry=str(stoichiometry), constant="true")

        # Create kinetic law with MathML
        kinetic_law = ET.SubElement(reaction, "kineticLaw")
        math = ET.SubElement(kinetic_law, "math", xmlns="http://www.w3.org/1998/Math/MathML")
        math.append(ET.fromstring(kinetic_law_math))  # Parse MathML from string and append



def create_model():
    
    # adj_matrix = AdjacencyMatrixGenerator.generate_adjacency_matrix()
    number_of_patches = len(adj_matrix.values[0])
    
    adjacency_matrix = adj_matrix.values
    
    # if model_type == "SIRS":
    sir_model = SIRModelSBML()
    
    def compute_theta_values(patch_num, neighbor_patch, distance_matrix):
        distance = distance_matrix[patch_num - 1, neighbor_patch - 1]

        if distance == 0:
            return 0, 0, 0  # No self-effect

        theta_S = 1 / (10 + distance**2)
        theta_I = 1 / (15 + distance**2)
        theta_R = 1 / (20 + distance**2)
        alpha_S = 1 / (25 + distance**2)
        alpha_I = 1 / (50 + distance**2)
        return theta_S, theta_I, theta_R, alpha_S, alpha_I
    
    vector_id = ["H", "M"]

    param_name = "sigma"
    param_id = f"{param_name}"
    param_value = "0.1"
    sir_model.add_parameter(param_id, name=param_name, value=param_value, constant="false")

    for param_name in ["mu"]:
        for v_id, value in [(vector_id[0], "0.01")]:
            param_id = f"{param_name}{v_id}"
            sir_model.add_parameter(param_id, name=param_name, value=value, constant="false")

    for patch_num in range(1, number_of_patches + 1):
        #patch_num = adj_matrix.columns[patch_num-1] # Error for np.where(adjacency_matrix[patch_num - 1] == 1
        compartment_id = f"compartment{patch_num}"
        sir_model.add_compartment(compartment_id, spatial_dimensions="3", size="1", constant="true")

        for species_type, initial_value in [("S", 4000), ("I", 20), ("R", 20), ("D_S", 0), ("D_I", 0), ("D_R", 0)]:
            species_id = f"{species_type}{vector_id[0]}{patch_num}"
            sir_model.add_species(species_id, name=species_id, compartment=compartment_id, initial_concentration=str(initial_value))

        for species_type, initial_value in [("S", 2000), ("I", 10), ("D_S", 0), ("D_I", 0)]:
            species_id = f"{species_type}{vector_id[1]}{patch_num}"
            sir_model.add_species(species_id, name=species_id, compartment=compartment_id, initial_concentration=str(initial_value))
            
        for param_name in ["beta"]:
            for v1, v2, value in [(vector_id[0], vector_id[1], "0.3"), (vector_id[1], vector_id[0], "0.4")]:
                param_id = f"{param_name}{v1}{v2}_{patch_num}_{patch_num}"
                sir_model.add_parameter(param_id, name=param_name, value=value, constant="false")
        
        for param_name in ["mu"]:
            for v_id, value in [(vector_id[1], "0.02")]:
                param_id = f"{param_name}{v_id}_{patch_num}"
                sir_model.add_parameter(param_id, name=param_name, value=value, constant="false")

        for state_var in ["S", "I", "R"]:
            initial_assignment_symbol = f"{state_var}{patch_num}"
            sir_model.add_initial_assignment(initial_assignment_symbol, f"<ci>{state_var}0</ci>")
            
        # reactions and kinetic laws WITHIN the patch
        kinetic_law_reaction_1 = f"<ci>MassAction(sigma)</ci>"
        kinetic_law_reaction_2 = f"<ci>MassAction(beta{vector_id[0]}{vector_id[1]}_{patch_num}_{patch_num})</ci>"
        kinetic_law_reaction_3 = f"<ci>MassAction(mu{vector_id[0]})</ci>"

        kinetic_law_reaction_4 = f"<ci>MassAction(beta{vector_id[1]}{vector_id[0]}_{patch_num}_{patch_num})</ci>"
        kinetic_law_reaction_5 = f"<ci>MassAction(mu{vector_id[1]}_{patch_num})</ci>"

        sir_model.add_reaction(f"I{vector_id[0]}_{patch_num}_R{vector_id[0]}", reactants={f"I{vector_id[0]}{patch_num}": 1}, products={f"R{vector_id[0]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_1)
        sir_model.add_reaction(f"S{vector_id[0]}_{patch_num}_I{vector_id[1]}", reactants={f"S{vector_id[0]}{patch_num}": 1, f"I{vector_id[1]}{patch_num}": 1} , products={f"I{vector_id[0]}{patch_num}": 1, f"I{vector_id[1]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_2)
        sir_model.add_reaction(f"R{vector_id[0]}_{patch_num}", reactants={f"R{vector_id[0]}{patch_num}": 1}, products={f"D_R{vector_id[0]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_3)
        sir_model.add_reaction(f"S{vector_id[0]}_{patch_num}", reactants={f"S{vector_id[0]}{patch_num}": 1}, products={f"D_S{vector_id[0]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_3)
        sir_model.add_reaction(f"I{vector_id[0]}_{patch_num}", reactants={f"I{vector_id[0]}{patch_num}": 1}, products={f"D_I{vector_id[0]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_3)

        sir_model.add_reaction(f"S{vector_id[1]}_{patch_num}_I{vector_id[0]}", reactants={f"S{vector_id[1]}{patch_num}": 1, f"I{vector_id[0]}{patch_num}": 1}, products={f"I{vector_id[1]}{patch_num}": 1, f"I{vector_id[0]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_4)
        sir_model.add_reaction(f"S{vector_id[1]}_{patch_num}", reactants={f"S{vector_id[1]}{patch_num}": 1}, products={f"D_S{vector_id[1]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_5)
        sir_model.add_reaction(f"I{vector_id[1]}_{patch_num}", reactants={f"I{vector_id[1]}{patch_num}": 1}, products={f"D_I{vector_id[1]}{patch_num}": 1}, kinetic_law_math=kinetic_law_reaction_5)

        # reactions and kinetic laws BETWEEN patches
        for neighbor_patch in np.where(adjacency_matrix[patch_num - 1] == 1)[0] + 1:
            theta_S, theta_I, theta_R, alpha_S, alpha_I = compute_theta_values(patch_num, neighbor_patch, distance_matrix)
            sir_model.add_parameter(f"theta_S{vector_id[0]}_{patch_num}{neighbor_patch}", name="theta_S", value=str(theta_S), constant="false")
            sir_model.add_parameter(f"theta_I{vector_id[0]}_{patch_num}{neighbor_patch}", name="theta_I", value=str(theta_I), constant="false")
            sir_model.add_parameter(f"theta_R{vector_id[0]}_{patch_num}{neighbor_patch}", name="theta_R", value=str(theta_R), constant="false")

            sir_model.add_parameter(f"alpha_S{vector_id[1]}_{patch_num}{neighbor_patch}", name="alpha_S", value=str(alpha_S), constant="false")
            sir_model.add_parameter(f"alpha_I{vector_id[1]}_{patch_num}{neighbor_patch}", name="alpha_I", value=str(alpha_I), constant="false")

            param_name = "beta"
            for v1, v2, value in [(vector_id[0], vector_id[1], "0.2"), (vector_id[1], vector_id[0], "0.25")]:
                param_id = f"{param_name}{v1}{v2}_{patch_num}_{neighbor_patch}"
                sir_model.add_parameter(param_id, name=param_name, value=value, constant="false")

            kinetic_law_reaction_between_patches_SH = f"<ci>MassAction(theta_S{vector_id[0]}_{patch_num}{neighbor_patch})</ci>"
            kinetic_law_reaction_between_patches_IH = f"<ci>MassAction(theta_I{vector_id[0]}_{patch_num}{neighbor_patch})</ci>"
            kinetic_law_reaction_between_patches_RH = f"<ci>MassAction(theta_R{vector_id[0]}_{patch_num}{neighbor_patch})</ci>"

            kinetic_law_reaction_between_patches_SM = f"<ci>MassAction(alpha_S{vector_id[1]}_{patch_num}{neighbor_patch})</ci>"
            kinetic_law_reaction_between_patches_IM = f"<ci>MassAction(alpha_I{vector_id[1]}_{patch_num}{neighbor_patch})</ci>"

            kinetic_law_reaction_between_patches_SHMII = f"<ci>MassAction(beta{vector_id[0]}{vector_id[1]}_{patch_num}_{neighbor_patch})</ci>"
            kinetic_law_reaction_between_patches_SMHII = f"<ci>MassAction(beta{vector_id[1]}{vector_id[0]}_{patch_num}_{neighbor_patch})</ci>"

            sir_model.add_reaction(f"S{vector_id[0]}_{patch_num}_{neighbor_patch}_S{vector_id[0]}", reactants={f"S{vector_id[0]}{patch_num}": 1}, products={f"S{vector_id[0]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_SH)
            sir_model.add_reaction(f"I{vector_id[0]}_{patch_num}_{neighbor_patch}_I{vector_id[0]}", reactants={f"I{vector_id[0]}{patch_num}": 1}, products={f"I{vector_id[0]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_IH)
            sir_model.add_reaction(f"R{vector_id[0]}_{patch_num}_{neighbor_patch}_R{vector_id[0]}", reactants={f"R{vector_id[0]}{patch_num}": 1}, products={f"R{vector_id[0]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_RH)

            sir_model.add_reaction(f"S{vector_id[1]}_{patch_num}_{neighbor_patch}_S{vector_id[1]}", reactants={f"S{vector_id[1]}{patch_num}": 1}, products={f"S{vector_id[1]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_SM)
            sir_model.add_reaction(f"I{vector_id[1]}_{patch_num}_{neighbor_patch}_I{vector_id[1]}", reactants={f"I{vector_id[1]}{patch_num}": 1}, products={f"I{vector_id[1]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_IM)

            sir_model.add_reaction(f"S{vector_id[0]}_{patch_num}_{neighbor_patch}_I{vector_id[1]}", reactants={f"S{vector_id[0]}{patch_num}": 1, f"I{vector_id[1]}{neighbor_patch}": 1}, products={f"I{vector_id[0]}{patch_num}": 1, f"I{vector_id[1]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_SHMII)
            sir_model.add_reaction(f"S{vector_id[1]}_{patch_num}_{neighbor_patch}_I{vector_id[0]}", reactants={f"S{vector_id[1]}{patch_num}": 1, f"I{vector_id[0]}{neighbor_patch}": 1}, products={f"I{vector_id[1]}{patch_num}": 1, f"I{vector_id[0]}{neighbor_patch}": 1}, kinetic_law_math=kinetic_law_reaction_between_patches_SMHII)

        tree = ET.ElementTree(sir_model.root)
        tree.write("SIRS_Mosq_2grid.xml", encoding="UTF-8", xml_declaration=True, method="xml")

create_model()

### Convert SBML model file to .andl

In [ ]:
def sbml_to_andl(input_xml: str, output_andl: str):
    """
    Runs: spike load -f=<input_xml> save -f=<output_andl>
    """
    cmd = [
        spike_path,
        "load", f"-f={input_xml}",
        "-net=SPN",
        "save", f"-f={output_andl}"
    ]
    try:
        subprocess.run(cmd, check=True)
        print(f"Converted {input_xml} -> {output_andl}")
    except subprocess.CalledProcessError as e:
        print("Spike conversion failed:", e)

spike_path = r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/spike"
sbml_to_andl("SIRS_Mosq_2grid.xml", "SIRS_Mosq_2grid.andl")

## Compute functional Paramters and Run Spike simulation

In [ ]:
MODEL_PATH = Path(r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/Simulated_data/SIRS_Mosq_2grid.andl")
OUTPUT_DIR = Path(r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/Simulated_data/Simulated_data")
SPIKE_EXE  = Path(r"/home/bkmanu/NIH/spike-1.6.0rc2-linux64/spike")

# number of underlying constant samples
N_samples = 1204

# Basis functions
def briere(T, a, Tmin, Tmax):
    if T <= Tmin or T >= Tmax:
        return 0.0
    return a * T * (T - Tmin) * math.sqrt(Tmax - T)

def logistic(RH, k, RHopt):
    return 1.0 / (1.0 + math.exp(-k * (RH - RHopt)))

def eyring(T, Psi_ad, AE_ad, R):
    Tk = T + 273.15
    return Psi_ad * Tk * math.exp(-AE_ad / (R * Tk))

def L_eff_multi(RH_min, RH_max, k, RHopt, n):
    """
    Approximate the 24h mean of logistic(RH) by sampling n points
    along a half-cosine diurnal curve between RH_min and RH_max.
    """
    ts = np.linspace(0, 1, n)  # fraction of day
    RHs = RH_min + (RH_max - RH_min) * 0.5 * (1 - np.cos(2 * np.pi * ts))
    return np.mean([logistic(rh, k, RHopt) for rh in RHs])

def hourly_avg_briere(Tmean, Tmin_obs, Tmax_obs, a, Tmin, Tmax):
    DTR = Tmax_obs - Tmin_obs
    hours = np.arange(24)
    T_hr = Tmean + (DTR / 2) * np.sin((2 * np.pi / 24) * hours - np.pi/2)
    vals = []
    for Th in T_hr:
        if Th <= Tmin or Th >= Tmax:
            vals.append(0.0)
        else:
            vals.append(a * Th * (Th - Tmin) * math.sqrt(Tmax - Th))
    return np.mean(vals)

def haversine_distance(coord1, coord2):
    R = 3959.0  # Earth radius in miles
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def compute_centroid_dist_matrix(grid):
    P = len(grid)
    D_cent = np.zeros((P, P))
    centroids = [(pt.y, pt.x) for pt in grid.geometry.centroid]
    for i in range(P):
        for j in range(i + 1, P):
            d = haversine_distance(centroids[i], centroids[j])
            D_cent[i, j] = d
            D_cent[j, i] = d
    return D_cent

def compute_boundary_dist_matrix(grid):
    P = len(grid)
    D_bound = np.zeros((P, P))
    for i in range(P):
        for j in range(i + 1, P):
            poly_i = grid.geometry.iloc[i]
            poly_j = grid.geometry.iloc[j]
            p_i, p_j = nearest_points(poly_i, poly_j)
            coord_i = (p_i.y, p_i.x)
            coord_j = (p_j.y, p_j.x)
            d = haversine_distance(coord_i, coord_j)
            D_bound[i, j] = d
            D_bound[j, i] = d
    return D_bound

def env_scaler(row, params):
    Tmean    = row['Temp_avg_C']
    Tmin_obs = row['Tmin_smooth']
    Tmax_obs = row['Tmax_smooth']
    # RH       = row['RH_avg']

    # Transmission Brière
    briere_avg = hourly_avg_briere(
        Tmean, Tmin_obs, Tmax_obs,
        params['a'], params['Tmin'], params['Tmax']
    )
    B_bar = briere_avg

    # Migration Brière (a/10)
    a_mig          = params['a'] / 10.0
    briere_avg_mig = hourly_avg_briere(
        Tmean, Tmin_obs, Tmax_obs,
        a_mig, params['Tmin'], params['Tmax']
    )
    B_bar_mig = briere_avg_mig

    # Humidity scaling
    # L_rh = logistic(RH, params['k'], params['RHopt'])
    RH_min = row['RH_min_smooth']
    RH_max = row['RH_max_smooth']
    L_rh   = L_eff_multi(RH_min, RH_max, params['k'], params['RHopt'], n=6)

    # Eyring mortality
    E_unscaled = eyring(Tmean, params['Psi_ad'], params['AE_ad'], params['R'])
    E_temp = E_unscaled

    return B_bar, B_bar_mig, L_rh, E_temp

def compute_within_patch_rates(env_df, params):
    """
    Returns DataFrame with columns:
      ['Day','Patch','B_bar','B_bar_mig','L_rh','E_temp','beta_HM','beta_MH','mu_M']
    """
    records = []
    for _, row in env_df.iterrows():
        day   = int(row['Day_cont'])
        patch = int(row['ID'])
        B_bar, B_bar_mig, L_rh, E_temp = env_scaler(row, params)

        # Within-patch β_HM:
        beta_HM_ii = (
            params['lambda0_HM']
            + params['lambda1_HM'] * B_bar
            + params['lambda2_HM'] * L_rh
            + params['lambda3_HM'] * B_bar * L_rh
        )
        # Within-patch β_MH:
        beta_MH_ii = (
            params['gamma0_MH']
            + params['gamma1_MH'] * B_bar
            + params['gamma2_MH'] * L_rh
            + params['gamma3_MH'] * B_bar * L_rh
        )
        # Mosquito mortality μ_M^(i,i):
        mu_M_ii = (
            params['p0_mortality']
            + params['p1_mortality'] * E_temp
        )

        rec = {
            'Day':     day,
            'Patch':   patch,
            'B_bar':   B_bar,
            'B_bar_mig': B_bar_mig,
            'L_rh':    L_rh,
            'E_temp':  E_temp,
            'beta_HM': beta_HM_ii,
            'beta_MH': beta_MH_ii,
            'mu_M':    mu_M_ii
        }
        records.append(rec)

    return pd.DataFrame.from_records(
        records,
        columns=[
            'Day','Patch','B_bar','B_bar_mig','L_rh','E_temp',
            'beta_HM','beta_MH','mu_M'
        ]
    )

def compute_theta_alpha(i, j, D_human, D_mosq, c_vals):
    d_h = D_human[i-1, j-1]
    theta_S = 1.0 / (c_vals['S'] + d_h**2)
    theta_I = 1.0 / (c_vals['I'] + d_h**2)
    theta_R = 1.0 / (c_vals['R'] + d_h**2)

    d_m = D_mosq[i-1, j-1]
    if d_m > 3.0:
        alpha_S_raw = 0.0
        alpha_I_raw = 0.0
    else:
        alpha_S_raw = 1.0 / (c_vals['alpha_S'] + d_m**2)
        alpha_I_raw = 1.0 / (c_vals['alpha_I'] + d_m**2)

    return theta_S, theta_I, theta_R, alpha_S_raw, alpha_I_raw

def compute_between_patch_rates(local_rates_df, D_human, D_mosq, c_vals, params):
    """
    Returns DataFrame with columns:
      ['Day','From','To','beta_HM','beta_MH','alpha_SM','alpha_IM'].
    """
    records = []
    P = D_human.shape[0]

    for _, row in local_rates_df.iterrows():
        day    = int(row['Day'])
        i      = int(row['Patch'])
        B_bar  = row['B_bar']
        B_bar_mig = row['B_bar_mig']
        L_rh   = row['L_rh']

        for j in range(1, P+1):
            if j == i:
                continue

            # Between-patch β_HM^(i,j):
            beta_HM_ij = (
                params['delta0_HM']
                + params['delta1_HM'] * B_bar
                + params['delta2_HM'] * L_rh
                + params['delta3_HM'] * B_bar * L_rh
            )
            # Between-patch β_MH^(i,j):
            beta_MH_ij = (
                params['eta0_MH']
                + params['eta1_MH'] * B_bar
                + params['eta2_MH'] * L_rh
                + params['eta3_MH'] * B_bar * L_rh
            )

            # Compute α‐kernels:
            theta_S, theta_I, theta_R, alpha_S_raw, alpha_I_raw = compute_theta_alpha(
                i, j, D_human, D_mosq, c_vals
            )
            alpha_S = alpha_S_raw * B_bar_mig
            alpha_I = alpha_I_raw * B_bar_mig

            rec = {
                'Day':     day,
                'From':    i,
                'To':      j,
                'beta_HM': beta_HM_ij,
                'beta_MH': beta_MH_ij,
                'alpha_SM': alpha_S,
                'alpha_IM': alpha_I
            }
            records.append(rec)

    return pd.DataFrame.from_records(
        records,
        columns=['Day','From','To','beta_HM','beta_MH','alpha_SM','alpha_IM']
    )

# write spc and run spike
def write_spc_file(spc_path: Path, model_file: Path, output_dir: Path, rates_df, sample_id: int):
    """Write an SPC that runs all days stepwise with onStep updates."""
    mf = model_file
    od = output_dir

    with open(spc_path, 'w', encoding='utf-8') as f:
        f.write('/**\n')
        f.write(' * Configuration of Vector Model\n')
        f.write(' */\n\n')

        # import block
        f.write('import: {\n')
        f.write(f'    from: "{mf}";\n')
        f.write('}\n\n')

        # configuration block
        f.write('configuration: {\n\n')

        # model constants
        f.write('  model: {\n')
        f.write('    constants: {\n')
        f.write('      parameter: {\n')
        f.write('      }\n')
        f.write('    }\n')
        f.write('    places: {\n')
        f.write('       // SH1: [[1000, 2000]];\n')
        f.write('    }\n')
        f.write('  }\n\n')

        # simulation 
        # T = len(rates_df)
        f.write('  simulation: {\n')
        f.write('    name: "SIRIS";\n')
        f.write('    type: [[\n')
        f.write('      stochastic: {\n')
        f.write('        solver: direct: {\n')
        f.write('           threads: 10;\n')
        f.write('           runs: 2;\n')
        f.write('        }\n')
        f.write('      }\n')
        f.write('    ]];\n')
        f.write('    interval: 0:365:365;\n\n')

        # Stepwise block
        f.write('    onStep: enabled: {\n')
        f.write('       do:{\n')
        for _, row in rates_df.iterrows():
            d = int(row['Day'])
            f.write(f'      if (simulation.step == {d}) {{ \n')
            for col in rates_df.columns.drop('Day'):
                v = row[col]
                f.write(f'        constant.{col} = {v:.6e};\n')
            f.write('      }\n')
        f.write('    }\n')
        f.write('  }\n\n')
        
        # export block
        filename = f"sample_{sample_id:02d}_stepwise"
        f.write('    export: {\n')
        f.write('      places: []; // all places\n')
        f.write('      transitions: []; // all transitions\n')
        f.write('      observers: [];\n')
        f.write('      csv: {\n')
        f.write('        sep: ","; // Separator\n')
        f.write(f'        file: "{od}\\{filename}"\n')
        f.write('           << ".csv";\n')
        f.write('      }\n')
        f.write('    }\n')
        f.write('  }\n')
        f.write('}\n\n')

        # log block
        f.write('log: {\n')
        f.write('  sim.varExa: configuration.simulation.type;\n')
        f.write('}')

def run_spike(spike_executable: Path, spc_path: Path) -> None:
    '''Write SPC, invoke Spike, then delete the file.'''
    cmd = f'"{spike_executable}" exe -f="{spc_path}"'
    print(f"[INFO] Running command: {cmd}")
    try:
        res = subprocess.run(cmd, check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("[INFO] Simulation completed successfully.")
        if res.stdout:
            print(res.stdout.decode(errors='ignore'))
        if res.stderr:
            print("[WARN] ", res.stderr.decode(errors='ignore'))
    except subprocess.CalledProcessError as e:
        print(f"[ERROR] Simulation failed (exit code {e.returncode}):")
        print(e.stderr.decode(errors='ignore'))
    finally:
        spc_path.unlink(missing_ok=True)
        print(f"[INFO] Removed temporary SPC file: {spc_path}")


# check output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# prepare and load data
env_df = grid_daily[grid_daily['Day_cont'] <= 365]
P      = len(grid1)                     
days   = sorted(env_df['Day_cont'].unique()) 
T_days = len(days)

# Pre‐compute distances & c_vals once
D_human = compute_centroid_dist_matrix(grid1)
D_mosq  = compute_boundary_dist_matrix(grid1)
c_vals  = {'S': 10, 'I': 15, 'R': 20, 'alpha_S': 25, 'alpha_I': 50}

# Build a list of rate‐column names
rate_col_names = ['Day']
for i in range(1, P+1):
    rate_col_names.append(f"betaHM_{i}_{i}")
    rate_col_names.append(f"betaMH_{i}_{i}")
    rate_col_names.append(f"muM_{i}")
for i in range(1, P+1):
    for j in range(1, P+1):
        if i == j:
            continue
        rate_col_names.append(f"betaHM_{i}_{j}")
        rate_col_names.append(f"betaMH_{i}_{j}")
        rate_col_names.append(f"alpha_SM_{i}{j}")
        rate_col_names.append(f"alpha_IM_{i}{j}")

# store all constants for each sample s
constants_records = []    
np.random.seed(42)  # for reproducibility

# Ross–Macdonald range
beta_min_HM, beta_max_HM = 0.01, 0.80   # mosquito → human
beta_min_MH, beta_max_MH = 0.072, 0.64  # human → mosquito
Delta_HM = beta_max_HM - beta_min_HM
Delta_MH = beta_max_MH - beta_min_MH

# Ross–Macdonald mosquito mortality (per day)
mu_min, mu_max = 0.05, 0.33
Delta_mu = mu_max - mu_min

for s in range(N_samples):
    print(f"\n=== Generating Sample {s} of {N_samples-1} ===")

    # Within‐patch HM
    lambda0_HM = beta_min_HM
    lambda1_HM = np.random.uniform(0, Delta_HM)
    lambda2_HM = np.random.uniform(0, Delta_HM)
    lambda3_HM = np.random.uniform(0, Delta_HM)

    # Within‐patch MH
    gamma0_MH = beta_min_MH
    gamma1_MH = np.random.uniform(0, Delta_MH)
    gamma2_MH = np.random.uniform(0, Delta_MH)
    gamma3_MH = np.random.uniform(0, Delta_MH)

    # Between‐patch HM
    delta0_HM = beta_min_HM
    delta1_HM = np.random.uniform(0, Delta_HM)
    delta2_HM = np.random.uniform(0, Delta_HM)
    delta3_HM = np.random.uniform(0, Delta_HM)

    # Between‐patch MH
    eta0_MH   = beta_min_MH
    eta1_MH   = np.random.uniform(0, Delta_MH)
    eta2_MH   = np.random.uniform(0, Delta_MH)
    eta3_MH   = np.random.uniform(0, Delta_MH)

    # Mosquito mortality
    p0_mortality = mu_min
    p1_mortality = np.random.uniform(0, Delta_mu)

    # Build a flat dict of constants for this sample s
    const_dict = {
    # within‐patch HM
    "lambda0_HM": lambda0_HM,
    "lambda1_HM": lambda1_HM,
    "lambda2_HM": lambda2_HM,
    "lambda3_HM": lambda3_HM,

    # within‐patch MH
    "gamma0_MH":  gamma0_MH,
    "gamma1_MH":  gamma1_MH,
    "gamma2_MH":  gamma2_MH,
    "gamma3_MH":  gamma3_MH,

    # between‐patch HM
    "delta0_HM":  delta0_HM,
    "delta1_HM":  delta1_HM,
    "delta2_HM":  delta2_HM,
    "delta3_HM":  delta3_HM,

    # between‐patch MH
    "eta0_MH":    eta0_MH,
    "eta1_MH":    eta1_MH,
    "eta2_MH":    eta2_MH,
    "eta3_MH":    eta3_MH,

    # mosquito mortality
    "p0_mortality": p0_mortality,
    "p1_mortality": p1_mortality
    }

    # Save this sample’s constants for constants.csv
    record = {'sample_id': s}
    record.update(const_dict)
    constants_records.append(record)

    # Build params_s for computing rates
    params_s = {
        'a':      2.71e-4,  'Tmin':   14.67,  'Tmax':   41.0,
        'k':      0.1,      'RHopt':  70.0,
        'Psi_ad': 13327.0,  'AE_ad':  53135.0,  'R': 8.314,

        'lambda0_HM':   lambda0_HM,
        'lambda1_HM':   lambda1_HM,
        'lambda2_HM':   lambda2_HM,
        'lambda3_HM':   lambda3_HM,

        'gamma0_MH':    gamma0_MH,
        'gamma1_MH':    gamma1_MH,
        'gamma2_MH':    gamma2_MH,
        'gamma3_MH':    gamma3_MH,

        'delta0_HM':    delta0_HM,
        'delta1_HM':    delta1_HM,
        'delta2_HM':    delta2_HM,
        'delta3_HM':    delta3_HM,

        'eta0_MH':      eta0_MH,
        'eta1_MH':      eta1_MH,
        'eta2_MH':      eta2_MH,
        'eta3_MH':      eta3_MH,

        'p0_mortality': p0_mortality,
        'p1_mortality': p1_mortality
    }

    # Compute within‐patch rates for all days 1..T_days
    local_rates_df_s = compute_within_patch_rates(env_df, params_s)

    # Compute between‐patch rates
    between_rates_df_s = compute_between_patch_rates(local_rates_df_s, D_human, D_mosq, c_vals, params_s)

    # Build a DataFrame rates_df
    rows = []
    for d in days:
        daily_dict = {'Day': d}
        # within‐patch
        df_loc_d = local_rates_df_s[local_rates_df_s['Day'] == d]
        for i in range(1, P+1):
            row_i = df_loc_d[df_loc_d['Patch'] == i].iloc[0]
            daily_dict[f"betaHM_{i}_{i}"] = row_i['beta_HM']
            daily_dict[f"betaMH_{i}_{i}"] = row_i['beta_MH']
            daily_dict[f"muM_{i}"]        = row_i['mu_M']

        # between‐patch
        df_betw_d = between_rates_df_s[between_rates_df_s['Day'] == d]
        for i in range(1, P+1):
            for j in range(1, P+1):
                if i == j: 
                    continue
                row_ij = df_betw_d[(df_betw_d['From'] == i) & (df_betw_d['To'] == j)].iloc[0]
                daily_dict[f"betaHM_{i}_{j}"]  = row_ij['beta_HM']
                daily_dict[f"betaMH_{i}_{j}"]  = row_ij['beta_MH']
                daily_dict[f"alpha_SM_{i}{j}"] = row_ij['alpha_SM']
                daily_dict[f"alpha_IM_{i}{j}"] = row_ij['alpha_IM']

        rows.append(daily_dict)

    rates_df = pd.DataFrame(rows, columns=rate_col_names)

    # Create a subfolder for this sample’s Spike outputs
    sample_folder = OUTPUT_DIR / f"sample_{s:02d}"
    sample_folder.mkdir(parents=True, exist_ok=True)

    with tempfile.NamedTemporaryFile(suffix=".spc", delete=False) as tmp:
        spc_file = Path(tmp.name)

    # Write one SPC that runs all days stepwise and run it
    write_spc_file(spc_file, MODEL_PATH, sample_folder, rates_df, sample_id=s)
    run_spike(SPIKE_EXE, spc_file)

    print(f"✅ Sample {s}: stepwise simulation complete ({len(rates_df)} days)")

# After all N_samples done, write out constants.csv
constants_df = pd.DataFrame(constants_records)
constants_df.to_csv(OUTPUT_DIR / "constants.csv", index=False)
print(f"\nAll done! Wrote constants.csv and Spike outputs under:\n  {OUTPUT_DIR}\n")